# Liste mit möglichen Parsern für die Doku

## HTML.parser Paket

https://www.educative.io/edpresso/what-is-the-html-parser-in-python

In [1]:
from html.parser import HTMLParser
class Parser(HTMLParser):
  # method to append the start tag to the list start_tags.
  def handle_starttag(self, tag, attrs):
    global start_tags
    start_tags.append(tag)
    # method to append the end tag to the list end_tags.
  def handle_endtag(self, tag):
    global end_tags
    end_tags.append(tag)
  # method to append the data between the tags to the list all_data.
  def handle_data(self, data):
    global all_data
    all_data.append(data)
  # method to append the comment to the list comments.
  def handle_comment(self, data):
    global comments
    comments.append(data)
start_tags = []
end_tags = []
all_data = []
comments = []
# Creating an instance of our class.
parser = Parser()
# Poviding the input.
HTMLFile = open("hier könnte ihre html stehen", "r", encoding = "utf-8")
parser.feed(HTMLFile.read())
print("start tags:", start_tags)
print("end tags:", end_tags)
print("data:", all_data)
print("comments", comments)

FileNotFoundError: [Errno 2] No such file or directory: 'hier könnte ihre html stehen'

In [2]:
data =data.replace("\n"," ")
print(data)

NameError: name 'data' is not defined

## Beautiful Soup

https://www.twilio.com/blog/web-scraping-and-parsing-html-in-python-with-beautiful-soup

Problem:
    
    * meisten Parser lesen eine HTML Seite aus dem Browser aus
    * Unsere lokale Webseite wird also nicht abgedeckt

In [3]:
import requests
from bs4 import BeautifulSoup


vgm_url = 'hier könnte ihre html stehen'
html_text = requests.get(vgm_url).text
soup = BeautifulSoup(html_text, 'html.parser')


InvalidSchema: No connection adapters were found for 'C:/Users/luido/OneDrive/Desktop/ECU-Test/Einfuehrung/Grundlagen/Einfuehrung.html'

### Beautiful soup local
    
    https://www.geeksforgeeks.org/how-to-parse-local-html-file-in-python/

In [2]:
# Importing BeautifulSoup class from the bs4 module
from bs4 import BeautifulSoup

# Opening the html file
HTMLFile = open("hier könnte ihre html stehen", "r", encoding = "utf-8")

In [3]:
# Reading the file
index = HTMLFile.read()

In [4]:
# Creating a BeautifulSoup object and specifying the parser
S = BeautifulSoup(index,'html.parser')

In [5]:
# Using the select-one method to find the second element from the li tag
Tag = S.select_one('li:nth-of-type(2)')

In [6]:
# Using the decompose method
Tag.decompose()

In [3]:
# Using the prettify method to modify the code
print(S.body.prettify())

NameError: name 'S' is not defined

In [8]:
print(S.head)

<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>Erster Start — ECU-TEST 2022.2 Anwenderhandbuch (in Arbeit)</title>
<link href="../_static/pygments.css" rel="stylesheet" type="text/css"/>
<link href="../_static/tt-style.css" rel="stylesheet" type="text/css"/>
<link href="../_static/graphviz.css" rel="stylesheet" type="text/css"/>
<link href="../_static/ECU-TEST-ICO.ico" rel="shortcut icon"/>
<!--[if lt IE 9]>
    <script src="../_static/js/html5shiv.min.js"></script>
  <![endif]-->
<script data-url_root="../" id="documentation_options" src="../_static/documentation_options.js"></script>
<script src="../_static/jquery.js"></script>
<script src="../_static/underscore.js"></script>
<script src="../_static/doctools.js"></script>
<script src="../_static/translations.js"></script>
<script src="../_static/js/theme.js"></script>
<link href="../genindex.html" rel="index" title="Stichwortverzeichnis"/>
<link href="../search.html" rel=

In [9]:
print(S.h1.get_text())

Erster Start


In [10]:
# Using the recursiveChildGenerator method to traverse the html file
for TraverseTags in S.recursiveChildGenerator():
  # Traversing the names of the tags
    if TraverseTags.name:
      # Printing the names of the tags
        print(TraverseTags.name)

html
head
meta
meta
title
link
link
link
link
script
script
script
script
script
script
link
link
link
link
body
div
nav
div
div
a
img
div
div
form
input
input
input
div
ul
li
a
ul
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
section
nav
i
a
div
div
div
ul
li
a
li
li
hr
div
div
div
span
h1
a
p
p
span
span
span
img
p
a
span
p
span
img
p
div
h2
a
p
span
p
span
p
span
img
p
span
img
p
span
span
span
img
p
span
img
p
p
img
p
div
h2
a
p
p
p
p
span
span
span
p
span
a
span
img
p
span
span
span
img
p
span
span
img
p
p
span
img
p
span
span
p
span
span
img
p
p
span
span
span
span
img
p
span
p
span
span
span
p
p
span
img
p
span
div
h2
a
p
span
p
p
span
p
span
p
img
span
img
p
span
img
p
img
div
p
p
p
span
span
a
span
span
p
span
img
p
span
p
span
span
span
span
img
div
p
p
span
span
p
span
span
p
img
p
div
h2
a
p
p
footer
div
a
span
a
span
hr
div
p
span
script
script


In [4]:
print(f'HTML: {S.ul}, name: {S.ul.name}, text: {S.ul.text}')

NameError: name 'S' is not defined

In [5]:
print(S.prettify())

NameError: name 'S' is not defined

In [6]:
all_link = []
for link in S.find_all('a'):
    all_link.append(link.get('href'))
    print(link.get('href'))

NameError: name 'S' is not defined

In [7]:
print(S.get_text())

NameError: name 'S' is not defined

In [8]:
print(S.p)

NameError: name 'S' is not defined

In [9]:
all_p = []
for text in S.find_all('p'):
    all_p.append(text.get_text())
    print(text.get_text())
print(all_p)

NameError: name 'S' is not defined

In [10]:
all_unter = []
for unterkap in S.find_all('h2'):
    all_unter.append(unterkap.get_text())
    print(unterkap.get_text())

NameError: name 'S' is not defined

In [11]:
all_bild=[]
for bild in S.find_all('img'):
    all_bild.append(bild.get('src'))

print(all_bild)

NameError: name 'S' is not defined

In [12]:
data_text = S.get_text()
print(data_text)

NameError: name 'S' is not defined

In [13]:
import re

data_text =re.sub(r'\n\s*\n', r'\n\n', S.get_text().strip(), flags=re.M)
print(data_text)

NameError: name 'S' is not defined

### Mögliches Speichern unserer Ausgelesenen Daten

In [14]:
import csv

header = ['title','unterkapitel', 'body', 'links','bilder']
data = [S.h1.get_text(),all_unter,all_p,all_link, all_bild]

with open ('test_file1.csv','w',encoding='utf-8') as f:
    writer = csv.writer(f)
    #write header
    writer.writerow(header)
    #write the data
    writer.writerow(data)

NameError: name 'S' is not defined

In [15]:
header = ['title','unterkapitel', 'body', 'links','bilder']
data = [S.h1.get_text(),all_unter,data_text,all_link, all_bild]

with open ('test_file2.csv','w',encoding='utf-8') as f:
    writer = csv.writer(f)
    #write header
    writer.writerow(header)
    #write the data
    writer.writerow(data)


NameError: name 'S' is not defined

### Ein bisschen befüllen der CSV

In [16]:
HTMLFile2 = open("hier könnte ihre hmtl stehen", "r", encoding = "utf-8")
index2 = HTMLFile2.read()
# Creating a BeautifulSoup object and specifying the parser
soup = BeautifulSoup(index2,'html.parser')
# Using the select-one method to find the second element from the li tag
Tag2 = soup.select_one('li:nth-of-type(2)')
# Using the decompose method
Tag2.decompose()

FileNotFoundError: [Errno 2] No such file or directory: 'hier könnte ihre hmtl stehen'

In [17]:
data_text2 =re.sub(r'\n\s*\n', r'\n\n', soup.get_text().strip(), flags=re.M)
print(data_text2)

NameError: name 'soup' is not defined

In [18]:
soup.table

NameError: name 'soup' is not defined

In [19]:
tabelle = []
table_body = soup.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    tabelle.append([ele for ele in cols if ele]) # Get rid of empty values
print(tabelle)

NameError: name 'soup' is not defined